In [2]:
import pyagrum as gum
import pyagrum.lib.notebook as gnb
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed
import os
import pandas as pd

In [3]:
os.chdir("/Volumes/GROUPS/HER/Onderzoek/Neurologie/VCI onderzoek/Malin Overmars/01_HARTBREIN/04_NETWORK")
#os.chdir("O:/Neurologie/VCI onderzoek/Malin Overmars/01_HARTBREIN/04_NETWORK")

In [4]:
# Load trained BN
bn_reduced = gum.loadBN("bn_reduced.bifxml")

In [5]:
bn_vars = pd.read_parquet("data/bn_vars.parquet", engine="pyarrow")

In [6]:
# Build the layer map: {1: [var1, var2], 2: [...], ..., 7: [...]}
layer_map = (
    bn_vars.groupby("LAYER")["VARIABLE NAME"]
    .apply(list)
    .to_dict()
)

In [7]:
# Define your ordered layers and assign them increasing color intensity
layer_order = [
    'L0 – Unmodifiable demographics',
    'L1 – Modifiable demograhpics / Lifestyle factors',
    'L2 – Cardiovascular risk factors',
    'L3 - Treatment status',
    'L4 – Potential disease process markers',
    'L5 - Imaging markers of neurovascular damage',
    'L6 – Current and previous cardiovascular diagnoses / Vascular interventions',
    'L7 - Functional status',
    'L8 – Outcomes'  # assumed typo in L7, but we keep both if present
]

# Create a color intensity map
layer_color_map = {
    name: i / (len(layer_order)-1)
    for i, name in enumerate(layer_order)
}

# Build the nodeColor dictionary
node_colors = {}
for layer_name, variables in layer_map.items():
    color_val = layer_color_map.get(layer_name, 0.5)  # fallback if unknown
    for var in variables:
        if var in bn_reduced.names():
            node_colors[var] = color_val

In [8]:
# Step 1: Prepare the raw labels directly from pyAgrum
variables = bn_reduced.names()

# Build the label options for each variable including 'None'
labels_dict = {var: ['None'] + list(bn_reduced[var].labels()) for var in variables}

# Step 2: Build interact parameters
interact_params = {'bn_reduced': fixed(bn_reduced)}
for var in variables:
    interact_params[var] = labels_dict[var]

# Step 3: Build the interactive function
@interact(**interact_params)
def evidence_impact(bn_reduced, **kwargs):
    # Build evidence dictionary, excluding variables where 'None' was selected
    evs = {var: val for var, val in kwargs.items() if val != 'None'}

    gnb.showInference(
        bn_reduced,
        evs=evs,
        size="10!",
        nodeColor=node_colors,
        cmapNode=plt.get_cmap("BuPu")
    )

interactive(children=(Dropdown(description='HIPPOCAMPUS_ICV_RATIO', options=('None', '(0.159529;0.251299[', '[…

In [9]:
net.show("interactive_bn.html")

NameError: name 'net' is not defined